In [ ]:
# EDA for Solar Energy Dataset (Malanville, Benin)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

# Set seaborn style for plots
sns.set_style('whitegrid')



# Load dataset
file_path = '../data/benin-malanville.csv'
df = pd.read_csv(file_path, encoding='cp1252', parse_dates=['Timestamp'], low_memory=False)

# Display dataset shape and columns
print(f"Data Shape: {df.shape}")
print("Columns:\n", df.columns)

# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

# Timestamp overview
print(f"Timestamp range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
print("Data frequency (seconds):\n", df['Timestamp'].diff().value_counts().head())

# Missing data check
missing_percent = df.isnull().mean() * 100
print("Columns with >5% missing values:\n", missing_percent[missing_percent > 5])

# Basic statistics
print("\nDescriptive Statistics:\n", df.describe())

# Outlier detection using Z-score
cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
subset = df[cols]

z_scores = np.abs(zscore(df[cols], nan_policy='omit'))
outliers = (z_scores > 3).sum(axis=1)
print(f"Number of rows with outliers in any column: {(outliers > 0).sum()}")

# Impute missing values using median
df[cols] = df[cols].apply(lambda x: x.fillna(x.median()))

# Drop rows with remaining missing values
df.dropna(inplace=True)
print(f"Data shape after cleaning: {df.shape}")

# Extract time features
df['Hour'] = df['Timestamp'].dt.hour
df['Month'] = df['Timestamp'].dt.month

# Line plot: Irradiance over time
plt.figure(figsize=(14, 6))
for col in ['GHI', 'DNI', 'DHI']:
    plt.plot(df['Timestamp'], df[col], label=col)
plt.title('Solar Irradiance over Time')
plt.xlabel('Time')
plt.ylabel('Irradiance (W/m²)')
plt.legend()
plt.tight_layout()
plt.show()

# Boxplots for outlier visualization
for col in cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot for {col}')
    plt.tight_layout()
    plt.show()

# Mean ModA and ModB by Cleaning status
if 'Cleaning' in df.columns and df['Cleaning'].notnull().any():
    grouped = df.groupby('Cleaning')[['ModA', 'ModB']].mean()
    if not grouped.empty:
        grouped.plot(kind='bar', figsize=(8, 5))
        plt.title('Average ModA & ModB by Cleaning Event')
        plt.ylabel('Module Output')
        plt.tight_layout()
        plt.show()
    else:
        print("Grouped data is empty. Skipping ModA/ModB plot.")
else:
    print("'Cleaning' column missing or contains only NaN. Skipping ModA/ModB analysis.")

# Correlation heatmap
plt.figure(figsize=(8, 6))
corr = df[['GHI', 'DNI', 'DHI', 'TModA', 'TModB']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix: Irradiance & Module Temperatures')
plt.tight_layout()
plt.show()

# Completion notice
print(f"\nEDA completed on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")